In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
import joblib

In [ ]:
# Define paths
complete_data_path = "data/complete_data/hunt4_complete.csv"
mi_data_path = "data/mi_data"
results_path = "data/results"
pred_path = "data/pred_values"

# Create directories if they don't exist
os.makedirs(results_path, exist_ok=True)
os.makedirs(pred_path, exist_ok=True)

# Define feature groups
categorical_vars = ["maritalstat", "education", "health", "satlife", "painintens", "smoking", "alcofreq", "worktype", "cost_category"]
binary_vars = ["sex", "armpain", "neckpain", "backuppain", "lumbarpain", "hiplegpain", "specconsult", "hospadmiss", "altcons", "headache", "workshift"]
continuous_vars = ["age", "bmi", "pulse", "bpsyst", "bpdia", "waistcirc", "hipcirc", "actindex", "hadsdep", "hadsanx", "icpc_morbidity_index"]

In [ ]:
# Parameter space for XGBoost
param_space = {
    "predictor__learning_rate": Real(0.0001, 0.1, prior="log-uniform"),
    "predictor__max_depth": Categorical([1, 2, 4, 8]),
    "predictor__n_estimators": Categorical([100, 500]),
    "predictor__gamma": Categorical([0, 1, 5, 10]),
    "predictor__min_child_weight": Categorical([1, 5, 10]),
    "predictor__colsample_bytree": Real(0.1, 0.8),
    "predictor__subsample": Categorical([0.1, 0.2, 0.3, 0.4, 0.5]),
    "predictor__reg_alpha": Categorical([0, 1, 5, 10, 15, 20]),
    "predictor__reg_lambda": Categorical([1, 5, 10, 15, 20])
}

# Step 1: Parameter Tuning on Complete Data
print("Performing Bayesian optimization for parameter tuning on complete data...")
complete_data = pd.read_csv(complete_data_path)


# Prepare dataset
ids = complete_data["lopenr"]  # ID column
fold = complete_data["fold"]  # Cluster variable
y_complete = complete_data["high_cost_point"]  # Outcome variable
X_complete = complete_data.drop(columns=["lopenr", "fold", "high_cost_point", "persistent"])

# Define the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(sparse_output=False, drop="first"), categorical_vars)
    ],
    remainder="passthrough"  # Keep numerical columns as is
)

# Define the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('predictor', XGBClassifier(objective="binary:logistic", random_state=42))
])

# Bayesian optimization
opt = BayesSearchCV(
    estimator=pipeline,
    search_spaces=param_space,
    n_iter=50,  # Number of parameter configurations to test
    cv=5,  # Cross-validation within the dataset
    scoring="roc_auc",
    random_state=42,
    verbose=3
)

opt.fit(X_complete, y_complete)
best_params = opt.best_params_

# Save the best parameters
print("Best Parameters from Complete Data:", best_params)
pd.DataFrame([best_params]).to_csv(os.path.join(results_path, "best_params_complete.csv"), index=False)


In [ ]:
print(complete_data.head())

In [ ]:
print(complete_data["high_cost_point"].value_counts())
print(complete_data["high_cost_point"].value_counts(normalize=True))

In [ ]:
# Load the best parameters
best_params_file = os.path.join(results_path, "best_params_complete.csv")
best_params = pd.read_csv(best_params_file).iloc[0].to_dict()

# Ensure integer parameters are correctly cast
for param in ['max_depth', 'n_estimators', 'min_child_weight']:
    if f'predictor__{param}' in best_params:
        best_params[f'predictor__{param}'] = int(best_params[f'predictor__{param}'])

# Define the pipeline again
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(sparse_output=False, drop="first", handle_unknown="ignore"), categorical_vars)
    ],
    remainder="passthrough"  # Keep numerical columns as is
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('predictor', XGBClassifier(objective="binary:logistic", random_state=42))
])

# Apply the best parameters to the pipeline
pipeline.set_params(**best_params)

# Initialize a list to store the average performance across datasets and all fold results
performance_results = []  # For xgboost_iecv_mean.csv
all_fold_results = []  # For storing fold-level results across datasets
fold_results_all = []

# Function to calculate calibration metrics
def calculate_calibration(y_true, y_pred):
    slope = np.polyfit(y_pred, y_true, 1)[0]
    citl = np.mean(y_true) - np.mean(y_pred)
    return slope, citl

# Loop through all imputed datasets
for i in range(1, 21): 
    print(f"Processing imputed dataset {i}/20...")
    file_path = os.path.join(mi_data_path, f"hunt4_mi{i}.csv")
    dataset = pd.read_csv(file_path)

    # Prepare dataset
    ids = dataset["lopenr"]
    fold = dataset["fold"]
    y = dataset["high_cost_point"]
    X = dataset.drop(columns=["lopenr", "fold", "high_cost_point", "persistent"])

    # Initialize storage for fold predictions and fold performance
    all_fold_predictions = []
    fold_results = []

    # Cross-validation
    for fold_num in range(1, 6): 
        print(f"  Fitting fold {fold_num}/5 for imputed dataset {i}...")
        train_idx = fold != fold_num
        val_idx = fold == fold_num

        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        # Fit and predict using the pipeline
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict_proba(X_val)[:, 1]

        # Collect predictions for this fold
        fold_predictions = pd.DataFrame({
            "ID": ids[val_idx],
            "True": y_val,
            "Predicted": y_pred,
            "Fold": [fold_num] * len(y_val),
        })
        all_fold_predictions.append(fold_predictions)

        # Calculate performance metrics
        auc = roc_auc_score(y_val, y_pred)
        calibration_slope, citl = calculate_calibration(y_val, y_pred)

        # Store fold performance
        fold_results.append({
            "Fold": fold_num,
            "AUC": auc,
            "Calibration Slope": calibration_slope,
            "CITL": citl,
            "Imputation": i,
        })

    # Combine predictions for the dataset
    all_fold_predictions_df = pd.concat(all_fold_predictions, ignore_index=True)
    all_fold_predictions_df.to_csv(
        os.path.join(pred_path, f"pred_values_mi{i}.csv"), index=False
    )

    # Save performance results for this dataset
    fold_results_df = pd.DataFrame(fold_results)
    fold_results_df.to_csv(
        os.path.join(results_path, f"xgboost_iecv_mi{i}.csv"), index=False
    )

    # Append fold results for global averaging
    fold_results_all.extend(fold_results)

    # Save the trained pipeline
    if i == 1:
        saved_model_path = "savedmodels"
        os.makedirs(saved_model_path, exist_ok=True)
        joblib.dump(pipeline, os.path.join(saved_model_path, "xgboost_pipeline.joblib"))

# Combine fold-level results across all datasets
fold_results_all_df = pd.DataFrame(fold_results_all)

# Compute overall averages across all folds and datasets
overall_avg = fold_results_all_df.groupby("Fold", as_index=False).mean(numeric_only=True)
if "Imputation" in overall_avg.columns:
    overall_avg = overall_avg.drop(columns=["Imputation"])
overall_avg.loc[len(overall_avg)] = fold_results_all_df.mean(numeric_only=True).to_dict()
overall_avg.loc[len(overall_avg) - 1, "Fold"] = "Overall Average"

# Save overall performance metrics
overall_avg.to_csv(os.path.join(results_path, "xgboost_iecv_mean.csv"), index=False)

print("CV and model saving completed!")


In [ ]:
# Path to the results file
results_file = "data/results/xgboost_iecv_mean.csv"

# Read and display the CSV
results_df = pd.read_csv(results_file, sep=",")
results_df